### Exemplo de processo que não alcança estado estável

Exemplo de processo M/M/2, com $\lambda = 1$ e $\mu = 0,2$.

Como $\rho = \frac{\lambda}{s\times \mu} = \frac{1}{0,4} >1$, é esperado que o processo não estabilize.

A fila explodirá. 

Altere o código e calcule o tempo médio de espera dos clientes que finalizaram o serviço.

In [ ]:
def estabelecimento (environment, nome_cliente, servidor):
    print (f"{nome_cliente} chega ao estabelecimento em {environment.now}") 
    
    with servidor.request() as req:
        yield req
        print (f"{nome_cliente} inicia o serviço em {environment.now}") 
        yield environment.timeout(random.expovariate(1/5)) 
        print (f"{nome_cliente} finaliza o serviço e sai do estabelecimento em {environment.now}")

def chegadas (environment):
    i=1 
    while True:
        yield environment.timeout(random.expovariate(1))
        environment.process(estabelecimento (environment, 'Cliente %d' % i, servidor))
        i+=1

env3 = simpy.Environment()
servidor = simpy.Resource(env3, capacity=2)
env3.process(chegadas(env3))
env3.run(until = 480)